Based on https://colab.research.google.com/drive/1Z1lbR_oTSaeodv9tTm11uEhOjhkUx1L4?usp=sharing

In [ ]:
EPOCHS = 1
BATCH_SIZE = 16
LEARNING_RATE =  0.00002
mixup = False
alpha = 0.8
gridsize = 1
USESAM = False
frozenlayers = 0
imgname = "ak064t1aaaff137"
explainability = False

In [ ]:
!pip install ipython-autotime
%load_ext autotime

import typing
import io
import os
import torch
import numpy as np
import cv2
import csv
import glob
import sys
import shutil
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import pathlib
import random
import statistics
import statistics
import torchvision
import math
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import WeightedRandomSampler
from torch.autograd import Variable
from datetime import datetime
from urllib.request import urlretrieve
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from torchvision.transforms import ToTensor


In [ ]:
def rollout(attentions, discard_ratio, head_fusion):
    result = torch.eye(attentions[0].size(-1))
    with torch.no_grad():
        for attention in attentions:
            if head_fusion == "mean":
                attention_heads_fused = attention.mean(axis=1)
            elif head_fusion == "max":
                attention_heads_fused = attention.max(axis=1)[0]
            elif head_fusion == "min":
                attention_heads_fused = attention.min(axis=1)[0]
            else:
                raise "Attention head fusion type Not supported"

            # Drop the lowest attentions, but
            # don't drop the class token
            flat = attention_heads_fused.view(attention_heads_fused.size(0), -1)
            _, indices = flat.topk(int(flat.size(-1)*discard_ratio), -1, False)
            indices = indices[indices != 0]
            flat[0, indices] = 0

            I = torch.eye(attention_heads_fused.size(-1))
            a = (attention_heads_fused + 1.0*I.to(device))/2
            a = a / a.sum(dim=-1)

            result = torch.matmul(a, result.to(device))
    
    # Look at the total attention between the class token,
    # and the image patches
    mask = result[0, 0 , 1 :]
    # In case of 224x224 image, this brings us from 196 to 14
    width = int(mask.size(-1)**0.5)
    mask = mask.reshape(width, width).cpu().numpy()
    mask = mask / np.max(mask)
    return mask    

In [ ]:

datasetpath = ""
datasetpath2x2 = ""
if gridsize == 1:
  shutil.copy(datasetpath, "/content/imagesFINAL2.zip")
elif gridsize == 2:
  shutil.copy(datasetpath2x2, "/content/imagesFINAL2.zip")

!ls

In [ ]:
shutil.unpack_archive("/content/imagesFINAL2.zip", "/content/imagesFINAL2","zip")

In [ ]:
lookup = {}
pspifilepath = ""
pspifolder = ""
if not os.path.exists (pspifilepath):
    for path, dirs, files in os.walk(pspifolder, topdown = True):
        files = [ fi for fi in files if fi.endswith(".txt")]
        for f in files:
            txt = open(os.path.join(path, f), "r")
            PSPI = int(float(txt.read()))
            name = f[0:-9]
            lookup[name] = PSPI

    with open(pspifilepath, 'wb') as f:
        pickle.dump([lookup], f)
else:
    with open(pspifilepath, 'rb') as f:
        lookup = pickle.load(f)

In [ ]:
try:
  del lookup[0]["tv095t2aeunaff001"]
  del lookup[0]["tv095t2aeunaff002"]
  del lookup[0]["tv095t2aeunaff003"]
  del lookup[0]["tv095t2aeunaff004"]
  del lookup[0]["tv095t2aeunaff005"]
  del lookup[0]["tv095t2aeunaff006"]
  del lookup[0]["tv095t2aeunaff007"]  
except:
  print("error")

if gridsize == 2:
  try:
    del lookup[0]["tv095t2aeunaff008"]
    del lookup[0]["tv095t2aeunaff009"]
    del lookup[0]["tv095t2aeunaff010"]
  except:
    print("error")

  kl = []
  for key, value in lookup[0].items():
    if "001" in key or  "002" in key or "003" in key:
      kl += [key]
      
  for k in kl:
    del lookup[0][k]

with open('mycsvfile.csv','w') as f:
    w = csv.writer(f)
    w.writerows(lookup[0].items())

In [ ]:

for root, dirs, files in os.walk("/content/imagesFINAL2", topdown=True):
    for name in files:
       shutil.move(os.path.join(root,name), "/content/imagesFINAL2/" + name)

for f in glob.glob('/content/imagesFINAL2/**', recursive=True):
    if not f.endswith(".png") and os.path.isfile(f):
        os.remove(f)


root = '/content/imagesFINAL2'
folders = list(os.walk(root, topdown= False))[0:]

for folder in folders:
    if not folder[2]:
        try:
            os.rmdir(folder[0])
        except:
            pass

In [ ]:



torch.use_deterministic_algorithms(True) 
def seed_everything(seed=42):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

seed_everything()
labels_frame = pd.read_csv("/content/mycsvfile.csv", header=None)
la = ["bn080", "mg101", "aa048", "jh043", "jh123"]
lb = ["dn124", "vw121", "fn059", "dr052", "ll042"]
lc = ["jk103", "jl047", "ch092", "jy115", "bg096"]
ld = ["bm049", "kz120", "th108", "tv095", "ib109"]
le = ["nm106", "mg066", "hs107", "gf097", "ak064"]

splits = [la,lb,lc,ld,le]
if explainability == True:
  splits = [la]
avgf1 = []
matrixlist = []

presults = []


for s in splits:
  os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"
  try:
    presults += [(poutput, plabels)]
  except:
    pass
  poutput = []
  plabels = []
  class PainData(Dataset):
    def __init__(self, mode):
      self.labels_frame = pd.read_csv("/content/mycsvfile.csv", header=None)
      self.mode = mode
      if mode == "test":
        self.df = labels_frame[labels_frame[0].str.startswith(tuple(s))]
      elif mode == "train":
        self.df = labels_frame[labels_frame[0].str.startswith(tuple(s)) == False]
      else:
        sys.exit("mode needs to be 'train' or 'test'")
      self.transform = transforms.Compose([
      # you can add other transformations in this list
      transforms.ToTensor()
      ])
      self.ps = len(self.df[self.df[1] == 0])
      self.ns = len(self.df) - self.ps

    def __len__(self):
      return len(self.df.index)

    def __getitem__(self, idx):
      f = self.df.iloc[idx].to_numpy()
      name = f[0]
      label = torch.zeros(2)
      if f[1] != 0:
        label[1] = 1
      else:
        label[0] = 1

      if self.mode == "train" and idx % 5 == 0 and idx > 0 and mixup == True and "mg101" not in f[0]:
          # Choose another image/label randomly
          mixup_idx = None
          f2 = None  
          while True:
              mixup_idx = random.randrange(0, len(self.df.index))
              f2 = self.df.iloc[mixup_idx].to_numpy()
              f2e = f2[1]
              fe = f[1]

              if fe != 0:
                fe = 1
              
              if f2e != 0:
                f2e = 1
              
              if (fe != f2e and f[0][0:5] == f2[0][0:5]):
                break
          
          name2 = f2[0]
          label2 = f2[1]
          mixup_label = torch.zeros(2)

          if f2[1] != 0:
            mixup_label[1] = 1
          else:
            mixup_label[0] = 1

          if gridsize == 2:
            image = self.transform(Image.open('/content/imagesFINAL2/' + name + "_mesh.mat.png.png"))
            mixup_image = self.transform(Image.open('/content/imagesFINAL2/' + name2 + "_mesh.mat.png.png"))
          else:
            image = self.transform(Image.open('/content/imagesFINAL2/' + name + "_mesh.mat.png"))
            mixup_image = self.transform(Image.open('/content/imagesFINAL2/' + name2 + "_mesh.mat.png"))
          # Select a random number from the given beta distribution
          # Mixup the images accordingly
          lam = np.random.beta(alpha, alpha)
          imagef = lam * image + (1 - lam) * mixup_image
          labelf = lam * label + (1 - lam) * mixup_label
          return imagef, labelf
      if gridsize == 2:
       return self.transform(Image.open('/content/imagesFINAL2/' + name + "_mesh.mat.png.png")), label 
      else:
       return self.transform(Image.open('/content/imagesFINAL2/' + name + "_mesh.mat.png")), label 




  train_ds = PainData("train")
  test_ds = PainData("test")


  def resize_pos_embed(posemb, posemb_new): # example: 224:(14x14+1)-> 384: (24x24+1)
      # Rescale the grid of position embeddings when loading from state_dict. Adapted from
      # https://github.com/google-research/vision_transformer/blob/00883dd691c63a6830751563748663526e811cee/vit_jax/checkpoint.py#L224
      ntok_new = posemb_new.shape[1]
      if True:
          posemb_tok, posemb_grid = posemb[:, :1], posemb[0, 1:]  # posemb_tok is for cls token, posemb_grid for the following tokens
          ntok_new -= 1
      else:
          posemb_tok, posemb_grid = posemb[:, :0], posemb[0]
      gs_old = int(math.sqrt(len(posemb_grid)))     # 14
      gs_new = int(math.sqrt(ntok_new))             # 24
      posemb_grid = posemb_grid.reshape(1, gs_old, gs_old, -1).permute(0, 3, 1, 2)  # [1, 196, dim]->[1, 14, 14, dim]->[1, dim, 14, 14]
      posemb_grid = F.interpolate(posemb_grid, size=(gs_new, gs_new), mode='bicubic') # [1, dim, 14, 14] -> [1, dim, 24, 24]
      posemb_grid = posemb_grid.permute(0, 2, 3, 1).reshape(1, gs_new * gs_new, -1)   # [1, dim, 24, 24] -> [1, 24*24, dim]
      posemb = torch.cat([posemb_tok, posemb_grid], dim=1)   # [1, 24*24+1, dim]
      return posemb

  !pip install -q git+https://github.com/huggingface/transformers


  from transformers import ViTModel
  from transformers.modeling_outputs import SequenceClassifierOutput
  from transformers import ViTConfig



  model = ViTModel.from_pretrained('google/vit-base-patch16-224', output_attentions = True) # load pretrained model
  state_dict = model.state_dict()


  config = ViTConfig.from_pretrained('google/vit-base-patch16-224', image_size=224 * gridsize, output_attentions = True)
  # new model with custom image_size
  model = ViTModel(config=config)
  # update state_dict
  new_state_dict = state_dict.copy()
  old_posemb = new_state_dict['embeddings.position_embeddings']

  if gridsize == 2:
    posemb_tok, posemb_grid = old_posemb[:, :1], old_posemb[0, 1:]  # posemb_tok is for cls token, posemb_grid for the following tokens
    posemb_grid = posemb_grid.reshape(1, 14, 14, -1).permute(0, 3, 1, 2)  # [1, 196, dim]->[1, 14, 14, dim]->[1, dim, 14, 14]
    posemb_grid2 = torch.cat((posemb_grid, posemb_grid), 3)
    posemb_grid2 = torch.cat((posemb_grid2, posemb_grid2), 2)
    posemb =  posemb_grid2.permute(0, 2, 3, 1).reshape(1, 28 * 28, -1)

    posemb = torch.cat([posemb_tok, posemb], dim=1)   # [1, 24*24+1, dim]
    old_posemb = posemb
  new_state_dict['embeddings.position_embeddings'] = old_posemb

  # equip new model with state_dict
  model.load_state_dict(new_state_dict)

  class ViTForImageClassification(nn.Module):
      def __init__(self, num_labels=1):
          super(ViTForImageClassification, self).__init__()
          self.vit = model
          self.dropout = nn.Dropout(0.1)
          self.classifier = nn.Linear(self.vit.config.hidden_size, num_labels)
          self.num_labels = num_labels

      def forward(self, pixel_values, labels, att = False):
          outputs = self.vit(pixel_values=pixel_values)
          output = self.dropout(outputs.last_hidden_state[:,0])
          logits = self.classifier(output)

          loss = None
          if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
          if loss is not None:
            if att:
              return logits, loss.item(), outputs.attentions
            else:
              return logits, loss.item()
          else:
             if att:
              return logits, None, outputs.attentions
             else:
              return logits, None



  class SAM(torch.optim.Optimizer):
      def __init__(self, params, base_optimizer, rho=0.05, adaptive=False, **kwargs):
          assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

          defaults = dict(rho=rho, adaptive=adaptive, **kwargs)
          super(SAM, self).__init__(params, defaults)

          self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
          self.param_groups = self.base_optimizer.param_groups
          self.defaults.update(self.base_optimizer.defaults)

      @torch.no_grad()
      def first_step(self, zero_grad=False):
          grad_norm = self._grad_norm()
          for group in self.param_groups:
              scale = group["rho"] / (grad_norm + 1e-12)

              for p in group["params"]:
                  if p.grad is None: continue
                  self.state[p]["old_p"] = p.data.clone()
                  e_w = (torch.pow(p, 2) if group["adaptive"] else 1.0) * p.grad * scale.to(p)
                  p.add_(e_w)  # climb to the local maximum "w + e(w)"

          if zero_grad: self.zero_grad()

      @torch.no_grad()
      def second_step(self, zero_grad=False):
          for group in self.param_groups:
              for p in group["params"]:
                  if p.grad is None: continue
                  p.data = self.state[p]["old_p"]  # get back to "w" from "w + e(w)"

          self.base_optimizer.step()  # do the actual "sharpness-aware" update

          if zero_grad: self.zero_grad()

      @torch.no_grad()
      def step(self, closure=None):
          assert closure is not None, "Sharpness Aware Minimization requires closure, but it was not provided"
          closure = torch.enable_grad()(closure)  # the closure should do a full forward-backward pass

          self.first_step(zero_grad=True)
          closure()
          self.second_step()

      def _grad_norm(self):
          shared_device = self.param_groups[0]["params"][0].device  # put everything on the same device, in case of model parallelism
          norm = torch.norm(
                      torch.stack([
                          ((torch.abs(p) if group["adaptive"] else 1.0) * p.grad).norm(p=2).to(shared_device)
                          for group in self.param_groups for p in group["params"]
                          if p.grad is not None
                      ]),
                      p=2
                )
          return norm

      def load_state_dict(self, state_dict):
          super().load_state_dict(state_dict)
          self.base_optimizer.param_groups = self.param_groups

  from transformers import ViTFeatureExtractor

  # Define Model
  model = ViTForImageClassification(2)    
  # Feature Extractor
  feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
  feature_extractor.size = 224 * gridsize
  # Adam Optimizer
  if USESAM == False:
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
  else:
    base_optimizer = torch.optim.Adam
    optimizer = SAM(model.parameters(), base_optimizer, lr=LEARNING_RATE)

  loss_func = nn.CrossEntropyLoss()
  # Use GPU if available  
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
  if torch.cuda.is_available():
      model.cuda() 
  
  fln = []
  for i in range(frozenlayers):
    fln += ["layer." + str(i) + "."]

  for name, param in model.named_parameters():
      a = False
      param.requires_grad = True
      if "output" in name:
          param.requires_grad = False

      for n in fln:
        if n in name:
            a = True
      if a == True:
        param.requires_grad = False
  

  ## Train the Model


  nopain = train_ds.ns
  pain = train_ds.ps
  weights = []
  for idx, t in enumerate(train_ds):
    try:
      if t[1][1] < 0.5:
        weights.append(nopain)
      else:
        weights.append(pain)
    except:
      pass


  p = 0
  sampler2 = data.WeightedRandomSampler(weights=weights, num_samples=len(train_ds), replacement=True)
  train_loader = data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=False,  num_workers=2, sampler=sampler2)

  model.train()

  # Train the model
  for epoch in range(EPOCHS):        
    for step, (x, y) in enumerate(train_loader):
      # Change input array into list with each batch being one element
      q = BATCH_SIZE
      while True:
          try:
              x = np.split(np.squeeze(np.array(x)), len(x))
              break
          except:
              q -= 1


      # Remove unecessary dimension
      for index, array in enumerate(x):
        x[index] = np.squeeze(array)
      # Apply feature extractor, stack back into 1 tensor and then convert to tensor
      x = torch.tensor(np.stack(feature_extractor(x)['pixel_values'], axis=0))
      # Send to GPU if available
      x, y  = x.to(device), y.to(device)
      b_x = Variable(x)   # batch x (image)
      b_y = Variable(y)   # batch y (target)
      # Feed through model
      output, loss = model(b_x, None)

      # Calculate loss
      if loss is None:
        if USESAM == True: 
          loss = loss_func(output, b_y)  # use this loss for any training statistics
          loss.backward()
          optimizer.first_step(zero_grad=True)
          output, loss = model(b_x, None)

          # second forward-backward pass
          loss = loss_func(output, b_y) # make sure to do a full forward pass
          loss.backward()
          optimizer.second_step(zero_grad=True)

        else:
          loss = loss_func(output, b_y)   
          optimizer.zero_grad()           
          loss.backward()                 
          optimizer.step()

      if step % 10 == 0:
        pass

    state = {
    'epoch': epoch,
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    }

    savepath = ""
    torch.save(state, savepath + str(epoch) + '.pt')





  eval_loader  = data.DataLoader(test_ds, batch_size=1, shuffle=True, num_workers=2) 
    
  yt = []
  yp = []
  for step, (x, y) in enumerate(eval_loader):
    with torch.no_grad():

      # Change input array into list with each batch being one element
      q = BATCH_SIZE
      while True:
          try:
              x = np.split(np.squeeze(np.array(x)), len(x))
              break
          except:
              q -= 1

      # Remove unecessary dimension
      for index, array in enumerate(x):
        x[index] = np.squeeze(array)
      # Apply feature extractor, stack back into 1 tensor and then convert to tensor
      x = torch.tensor(np.stack(feature_extractor(x)['pixel_values'], axis=0))
      # Send to GPU if available
      x, y  = x.to(device), y.to(device)
      b_x = Variable(x)   # batch x (image)
      b_y = Variable(y)   # batch y (target)
      # Feed through model
      output, _ = model(b_x, None)
      poutput += [output]
      plabels += [y]
      predicted_class = output.argmax(1)

      for n in y.tolist():
        if n [0] == 1:
          yt += [0]
        else:
          yt += [1]
      yp += predicted_class.tolist()

  cf_matrix = confusion_matrix(yt, yp)
  matrixlist += [cf_matrix]
  print(cf_matrix)
  tn, fp, fn, tp = cf_matrix.ravel()
  f1score = 2 * tp / (2*tp + fp + fn)
  print("f1score: ", f1score)
  avgf1 += [f1score]

presults += [(poutput, plabels)]




In [ ]:
if explainability == True:


  eval_loader  = data.DataLoader(test_ds, batch_size=1, shuffle=True, num_workers=2)
  model.eval()
  th2 = 0.7
  output_dir = ''
  os.makedirs(output_dir, exist_ok=True)
  convert_tensor = transforms.Compose([
        transforms.ToTensor()
        ])
  if gridsize == 1:
    x = convert_tensor(Image.open('/content/imagesFINAL2/' + imgname	 + "_mesh.mat.png")).unsqueeze(0)
  if gridsize == 2:
    x = convert_tensor(Image.open('/content/imagesFINAL2/' + imgname	 + "_mesh.mat.png.png")).unsqueeze(0)

  with torch.no_grad():
    # Change input array into list with each batch being one element
    q = BATCH_SIZE
    while True:
        try:
            x = np.split(np.squeeze(np.array(x)), len(x))
            break
        except:
            q -= 1

    # Remove unecessary dimension
    for index, array in enumerate(x):
      x[index] = np.squeeze(array)
    # Apply feature extractor, stack back into 1 tensor and then convert to tensor
    x = torch.tensor(np.stack(feature_extractor(x)['pixel_values'], axis=0))
    # Send to GPU if available
    x, y  = x.to(device), y.to(device)
    b_x = Variable(x)   # batch x (image)
    b_y = Variable(y)   # batch y (target)
    # Feed through model
    outputs = model(b_x, None, True)
    attentions = outputs[2][-1]
    nh = 12 # number of head

    # we keep only the output patch attention
    attentions = attentions[0, :, 0, 1:].reshape(nh, -1)

    threshold = 0
    w_featmap = x.shape[-2] // 16
    h_featmap = x.shape[-1] // 16

    # we keep only a certain percentage of the mass
    val, idx = torch.sort(attentions)
    val /= torch.sum(val, dim=1, keepdim=True)
    cumval = torch.cumsum(val, dim=1)
    th_attn = cumval > (1 - threshold)
    idx2 = torch.argsort(idx)
    for head in range(nh):
        th_attn[head] = th_attn[head][idx2[head]]
    th_attn = th_attn.reshape(nh, w_featmap, h_featmap).float()
    # interpolate
    th_attn = nn.functional.interpolate(th_attn.unsqueeze(0), scale_factor=16, mode="nearest")[0].cpu().numpy()

    attentions = attentions.reshape(nh, w_featmap, h_featmap)
    attentions = nn.functional.interpolate(attentions.unsqueeze(0), scale_factor=16, mode="nearest")[0].cpu()
    attentions = attentions.detach().numpy()

    # show and save attentions heatmaps
    os.makedirs(output_dir, exist_ok=True)
    torchvision.utils.save_image(torchvision.utils.make_grid(x, normalize=True, scale_each=True), os.path.join(output_dir, s[0] + "gridsize" + str(gridsize) + "lr" + str(LEARNING_RATE) + "layers" + str(12-frozenlayers) + "img" + ".png"))

    if gridsize == 1:
      fff = np.zeros((224,224, 4))
    if gridsize == 2: 
      fff = np.zeros((448,448, 4))

    img = []

    for j in range(nh):
        fname = os.path.join(output_dir, s[0] + "gridsize" + str(gridsize) + "lr" + str(LEARNING_RATE) + "layers" + str(12-frozenlayers) + "attn-head" + str(j) + ".png")
        plt.figure()
        plt.imsave(fname=fname, arr=attentions[j], format='png')
        image_file = Image.open(fname)
        image_file = image_file.point( lambda p: p if p > th2 * 255 else 0 )
        fname = os.path.join(output_dir, "thresholded" + s[0] + "gridsize" + str(gridsize) + "lr" + str(LEARNING_RATE) + "layers" + str(12-frozenlayers) + "attn-head" + str(j) + ".png")

        image_file.save(fname)
        if img == []:
          img = np.array(image_file)
        else:
          img[0:3] = np.maximum(img, np.array(image_file))[0:3]

        fff += image_file
        
    fff /= 12 * 255
    plt.imshow(fff)
    plt.colorbar()
    plt.set_cmap('jet')
    fname = os.path.join(output_dir, "meanLL" + s[0] + "gridsize" + str(gridsize) + "lr" + str(LEARNING_RATE) + "layers" + str(12-frozenlayers) + "attn-head" + str(j) + ".png")
    plt.imsave(fname=fname, arr=fff, format='png')
    fname = os.path.join(output_dir, "maxLL" + s[0] + "gridsize" + str(gridsize) + "lr" + str(LEARNING_RATE) + "layers" + str(12-frozenlayers) + "attn-head" + str(j) + ".png")
    plt.imsave(fname=fname, arr=img, format='png')

  if gridsize == 1:
    x = convert_tensor(Image.open('/content/imagesFINAL2/' + imgname	 + "_mesh.mat.png")).unsqueeze(0)
  if gridsize == 2:
    x = convert_tensor(Image.open('/content/imagesFINAL2/' + imgname	 + "_mesh.mat.png.png")).unsqueeze(0)
  with torch.no_grad():
    if gridsize == 1:
      x = F.interpolate(x, size=(224,224), mode='bicubic')
    else:
      x = F.interpolate(x, size=(448,448), mode='bicubic')
    outputs = model(x.to(device), None, True)
    attentions = outputs[2]
    mask = rollout(attentions, 0.9, 'max')

    fname = os.path.join(output_dir,  "max" + s[0] + "gridsize" + str(gridsize) + "lr" + str(LEARNING_RATE) + "layers" + str(12-frozenlayers) + ".png")
    if gridsize == 1:
      res = cv2.resize(mask, dsize=(224, 224), interpolation=cv2.INTER_NEAREST)
    else:
      res = cv2.resize(mask, dsize=(448, 448), interpolation=cv2.INTER_NEAREST)
    plt.imshow(res)
    plt.colorbar()
    plt.set_cmap('jet')
    plt.imsave(fname=  fname, arr=res, format='png')
    image_file = Image.open(fname)
    image_file = image_file.point( lambda p: p if p > th2 * 255 else 0 )
    fname = os.path.join(output_dir, "thresholded" + "max" + s[0] + "gridsize" + str(gridsize) + "lr" + str(LEARNING_RATE) + "layers" + str(12-frozenlayers) + "attn-head" + str(j) + ".png")
    image_file.save(fname)

    mask = rollout(attentions, 0, 'mean')
    fname = os.path.join(output_dir,  "mean" + s[0] + "gridsize" + str(gridsize) + "lr" + str(LEARNING_RATE) + "layers" + str(12-frozenlayers) + ".png")
    if gridsize == 1:
      res = cv2.resize(mask, dsize=(224, 224), interpolation=cv2.INTER_NEAREST)
    else:
      res = cv2.resize(mask, dsize=(448, 448), interpolation=cv2.INTER_NEAREST)
    plt.imshow(res)
    plt.colorbar()
    plt.set_cmap('jet')
    plt.imsave(fname=  fname, arr=res, format='png')
    image_file = Image.open(fname)
    image_file = image_file.point( lambda p: p if p > th2 * 255 else 0 )
    fname = os.path.join(output_dir, "thresholded" + "mean" + s[0] + "gridsize" + str(gridsize) + "lr" + str(LEARNING_RATE) + "layers" + str(12-frozenlayers) + "attn-head" + str(j) + ".png")
    image_file.save(fname)
    fname = os.path.join(output_dir, s[0] + "gridsize" + str(gridsize) + "lr" + str(LEARNING_RATE) + "layers" + str(12-frozenlayers) + ".png")
    torchvision.utils.save_image(torchvision.utils.make_grid(x, normalize=True, scale_each=True), fname)





In [ ]:

e = []
f = []
tqtq = []

for n in presults:
  e = []
  f = []
  for a,b in zip(n[0], n[1]):
    for c,d in zip(a,b):
      c = c.cpu() 
      d = d.cpu()
      softmax = np.exp(c[1])/sum(np.exp(c))
      e += [softmax]
      f += [d.cpu()[1]]
  tqtq += [roc_auc_score(f, e)]
  print(roc_auc_score(f, e))
    
    
print(statistics.mean(tqtq))

In [ ]:

for m in matrixlist:
  print(m, end ="\t")
print("\nf1 scores "  , avgf1)
print("Mean f1 score ", statistics.mean(avgf1))
print("STD f1 score ", statistics.stdev(avgf1))
print("Layers used ", 12 - frozenlayers)
print("Mixup ", mixup)
if mixup == True:
  print("Alpha ", alpha)
print("learning rate ", LEARNING_RATE)
print("Using SAM ", USESAM)
print("Gridsize used ", gridsize)